In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss

In [2]:
import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.optimizers import RMSprop, Adam
from keras import backend as K

In [3]:
ROWS = 64 
COLS = 64
CHANNELS = 3

In [4]:
TRAIN_DIR="/content/drive/MyDrive/Colab Notebooks/Signature_Recognition_DeepLearning-master/newData/train/"

TEST_DIR='/content/drive/MyDrive/Colab Notebooks/Signature_Recognition_DeepLearning-master/newData/test/'

In [5]:
def get_images(fish):
    """Load files from train folder"""
    fish_dir = TRAIN_DIR+'{}'.format(fish)
    images = [fish+'/'+im for im in os.listdir(fish_dir)]
    return images

In [6]:
SIGNATURE_CLASSES = ['A', 'B', 'C','D','E']

In [7]:
files = []
y_all = []

for fish in SIGNATURE_CLASSES:
    fish_files = get_images(fish)
    files.extend(fish_files)
    
    y_fish = np.tile(fish, len(fish_files))
    y_all.extend(y_fish)
    print("{0} photos of {1}".format(len(fish_files), fish))
    
y_all = np.array(y_all)


5 photos of A
5 photos of B
4 photos of C
2 photos of D
2 photos of E


In [8]:
print(len(files))
print(len(y_all))

18
18


In [9]:
import cv2
def read_image(src):
    filepath=src
    im = cv2.imread(filepath)

    return cv2.resize(im,(ROWS,COLS))

In [10]:
X_all = np.ndarray((len(files), ROWS, COLS, CHANNELS), dtype=np.uint8)

for i, im in enumerate(files): 
    X_all[i] = read_image(TRAIN_DIR+im)
    if i%1000 == 0: print('Processed {} of {}'.format(i, len(files)))

print(X_all.shape)
# One Hot Encoding Labels
y_all = LabelEncoder().fit_transform(y_all)
y_all = np_utils.to_categorical(y_all)

Processed 0 of 18
(18, 64, 64, 3)


In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(X_all, y_all, test_size=0.1, random_state=0)

In [12]:
#Initialize the model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(ROWS, COLS, CHANNELS)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(96, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(96, (3, 3), activation='relu'))
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(1028, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

In [13]:
model.add(Dense(len(SIGNATURE_CLASSES), activation='softmax'))
# model.add(Dense(len(SIGNATURE_CLASSES), activation='sigmoid'))

In [14]:
model.compile(optimizer='adam', loss='categorical_crossentropy')
# adam = Adam(lr=0.0001)
# model.compile(optimizer=adam, loss=root_mean_squared_error)

In [15]:
early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1, mode='auto') 

In [16]:
model.fit(X_train, y_train, batch_size=64, epochs=25, validation_split=0.1, verbose=1, shuffle=True, callbacks=[early_stopping])

Epoch 1/25
1/1 [==============================] - 0s 288ms/step - loss: 16.2274 - val_loss: 1.9678
Epoch 2/25
1/1 [==============================] - 0s 37ms/step - loss: 6.4169 - val_loss: 1.7828
Epoch 3/25
1/1 [==============================] - 0s 46ms/step - loss: 3.2397 - val_loss: 1.7149
Epoch 4/25
1/1 [==============================] - 0s 32ms/step - loss: 2.1391 - val_loss: 1.6784
Epoch 5/25
1/1 [==============================] - 0s 37ms/step - loss: 1.7821 - val_loss: 1.6620
Epoch 6/25
1/1 [==============================] - 0s 41ms/step - loss: 2.1730 - val_loss: 1.6757
Epoch 7/25
1/1 [==============================] - 0s 39ms/step - loss: 2.0516 - val_loss: 1.6825
Epoch 8/25
1/1 [==============================] - 0s 38ms/step - loss: 1.9556 - val_loss: 1.6884
Epoch 9/25
1/1 [==============================] - 0s 43ms/step - loss: 1.7365 - val_loss: 1.6809
Epoch 00009: early stopping


In [17]:
from sklearn.metrics import log_loss
preds = model.predict(X_valid)
print("Validation Log Loss: {}".format(log_loss(y_valid, preds)))

Validation Log Loss: 1.5140101313591003


In [18]:
test_files = [im for im in os.listdir(TEST_DIR)]
test = np.ndarray((len(test_files), ROWS, COLS, CHANNELS), dtype=np.uint8)

for i, im in enumerate(test_files): 
    test[i] = read_image(TEST_DIR+im)
    
test_preds = model.predict(test)
submission = pd.DataFrame(test_preds, columns=SIGNATURE_CLASSES)
submission.insert(0, 'image', test_files)
submission.head()

,image,A,B,C,D,E
0,1.png,0.221390,0.216408,0.199678,0.183331,0.179192
1,2.png,0.220238,0.214072,0.201035,0.183614,0.181042
